In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/synthetic_data.db'
mask_file = '/data/zulqarnain/synthetic_data_with_condition/nifti_files/wholebrain.nii.gz'

OVERWRITE = True

In [4]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

In [5]:
validation_subjects = [6, 7, 13, 14]

In [6]:
if not existed or OVERWRITE:
    data_files = utils.sorted_glob('/data/zulqarnain/synthetic_data_with_condition/nifti_files/*.nii')
    regex = re.compile("subject_([0-9]{1,2})_condition_([1-2])_task_([1-2])_stimulus_([1-3]).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        [subject, condition, task, stimulus] = m.groups()
        subject = int(subject)
        condition = int(condition)
        task = int(task)
        stimulus = int(stimulus)
        logging.info("Recording blocks in %s", data_file)
        
        synthetic_block = niidb.FMriActivationBlock(zscore=False)
        synthetic_block.task = stimulus
        synthetic_block.filename = data_file
        synthetic_block.subject = subject
        synthetic_block.start_time = 0
        synthetic_block.end_time = None
        synthetic_block.individual_differences = {
            'condition': condition,
            'stimulus_group': task,
            'validation': subject in validation_subjects,
        }
        db.upsert(synthetic_block)
        logging.info('Subject %d performing task %s', subject, synthetic_block.task)

01/18/2019 17:49:06 Recording blocks in /data/zulqarnain/synthetic_data_with_condition/nifti_files/subject_3_condition_1_task_2_stimulus_2.nii
01/18/2019 17:49:06 Subject 3 performing task 2
01/18/2019 17:49:06 Recording blocks in /data/zulqarnain/synthetic_data_with_condition/nifti_files/subject_11_condition_2_task_2_stimulus_1.nii
01/18/2019 17:49:06 Subject 11 performing task 1
01/18/2019 17:49:06 Recording blocks in /data/zulqarnain/synthetic_data_with_condition/nifti_files/subject_6_condition_1_task_1_stimulus_2.nii
01/18/2019 17:49:06 Subject 6 performing task 2
01/18/2019 17:49:06 Recording blocks in /data/zulqarnain/synthetic_data_with_condition/nifti_files/subject_9_condition_2_task_1_stimulus_3.nii
01/18/2019 17:49:06 Subject 9 performing task 3
01/18/2019 17:49:06 Recording blocks in /data/zulqarnain/synthetic_data_with_condition/nifti_files/subject_3_condition_1_task_1_stimulus_3.nii
01/18/2019 17:49:06 Subject 3 performing task 3
01/18/2019 17:49:06 Recording blocks in /da

In [7]:
logging.info('Finished building NiiDb out of synthetic dataset')

01/18/2019 17:49:07 Finished building NiiDb out of synthetic dataset
